In [33]:
import os
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import geopy
from geopy.geocoders import Nominatim

In [34]:
#l'utilisation de variables d'environnement pour des raisons de sécurité 
#using environment variables for security reasons 
user_name = os.environ.get("DB_NAME")
password = os.environ.get("DB_PASSWORD")

In [35]:
#connexion à mysql
#connecting to mysql

host = 'localhost'
user = user_name
password = password 
db_name = 'dataengineer'
port = 3306

sqlEngine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}', pool_recycle=port)
dbconnection = sqlEngine.connect()


In [25]:
#requête sql pour importer la table d'adresses de la base de données dataengineer
#sql query to import the address table from dataengineer database

sql = """ SELECT * 
                FROM dataengineer.address"""

In [26]:
#lire la table dans un dataframe pandas
#reading the table into a pandas dataframe

df = pd.read_sql( sql, dbconnection)

In [27]:
# vérification des valeurs nulles dans le cadre de données
# checking for null values in the dataframe
df.isnull().sum()

address_id     0
address        0
city           0
postal_code    0
dtype: int64

In [28]:
df

,address_id,address,city,postal_code
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217
3,4,6 BD DES ETINES,LE COTEAU,42120
4,5,35 AV DU 159EME RIA,BRIANCON,5100
...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100
558,602,6 AV JEAN JAURES,FEURS,42110
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800


In [8]:
#This function adds the address + city + postal_code  of each row, and checks if it is available in 
#Openstreetmap api if its not, it adds the city + postal_code and checks the Openstreetmap api again, 
#then if it still not a valid address, it uses only the postal code, 
#this is done in order not to have null values for locations with incomplete information, 
#so we'll return the nearest latitude and longitude to the location 

#Cette fonction ajoute l'adresse + la ville + le code postal de chaque ligne, et vérifie si elle est disponible 
#dans l'api Openstreetmap, si ce n'est pas le cas, elle ajoute la ville + le code postal et vérifie à nouveau 
#l'api Openstreetmap, puis si ce n'est toujours pas une adresse valide, elle utilise seulement le code postal, 
#ceci est fait afin de ne pas avoir de valeurs nulles pour les lieux avec des informations incomplètes, 
#nous allons donc retourner la latitude et la longitude les plus proches du lieu. 
#de la localisation de ceux qui ont une mauvaise adresse.

def addition(address, city, postal_code):
    add = address + ' ' + city + ' ' + postal_code
    coordinates = Nominatim(user_agent='deen').geocode(add, timeout=20)
    
    if (coordinates != None):
        return add
    
    elif(coordinates == None):
        add = city + ' ' + postal_code
        coordinates = Nominatim(user_agent='deen').geocode(add, timeout=20)
        if (coordinates != None):
            return add   
    else:
        add = postal_code
        return add
 

In [9]:
#fonction qui appelle l'adresse openstreetmap adresse Api 
#function that calls the openstreetmap address Api address 
def addressLOCATION(add):
    coordinates = Nominatim(user_agent='deen').geocode(add, timeout=20)
    if (coordinates != None):
        return coordinates


In [10]:
df['add']  = df.apply(lambda x: addition(x['address'], x['city'], x['postal_code']), axis=1)

In [11]:
df['add'].isnull().sum()

0

In [12]:
df['add'].head(10)

0                318 CHE DE ROUMAGOUA LA CIOTAT 13600
1                19 RUE DES DAMES SAINTE SAVINE 10300
2            22 RUE AMIRAL GUEPRATTE LE CONQUET 29217
3                     6 BD DES ETINES LE COTEAU 42120
4                                       BRIANCON 5100
5                    36 RUE D EN HAUT BRUNEMONT 59151
6             1625 RTE DE CHAMBERY SAINT-ISMIER 38330
7          3 RUE MONTESQUIEU ASNIERES-SUR-SEINE 92600
8                                       ORLEANS 45000
9    67 BD COMMANDANT CHARCOT NEUILLY-SUR-SEINE 92200
Name: add, dtype: object

In [13]:
df['coordinates']  = df['add'].apply(lambda x: addressLOCATION(x))

In [14]:
#Using openstreetmap api to get the lungitude and latitude 
#Utilisation de l'api openstreetmap pour obtenir la lungitude et la latitude 

df['latitude'] = df['coordinates'].apply(lambda x: x.latitude if x != None else None)
df['longitude'] = df['coordinates'].apply(lambda x: x.longitude if x != None else None)

In [15]:
df

,address_id,address,city,postal_code,add,coordinates,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,318 CHE DE ROUMAGOUA LA CIOTAT 13600,"(Chemin de Roumagoua, La Fieloula, La Ciotat, ...",43.197927,5.607635
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,19 RUE DES DAMES SAINTE SAVINE 10300,"(Rue des Dames, Sainte-Savine, Troyes, Aube, G...",48.290905,4.045789
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,22 RUE AMIRAL GUEPRATTE LE CONQUET 29217,"(Rue Amiral Guépratte, Le Croaë, Le Conquet, B...",48.361340,-4.766800
3,4,6 BD DES ETINES,LE COTEAU,42120,6 BD DES ETINES LE COTEAU 42120,"(Boulevard des Etines, Le Coteau, Roanne, Loir...",46.021659,4.092120
4,5,35 AV DU 159EME RIA,BRIANCON,5100,BRIANCON 5100,"(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",44.898404,6.643631
...,...,...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,20 RUE COLI MONTREUIL 93100,"(20, Rue Coli, Ruffins - Théophile-Sueur, Mont...",48.868459,2.468415
558,602,6 AV JEAN JAURES,FEURS,42110,6 AV JEAN JAURES FEURS 42110,"(6, Avenue Jean Jaurès, Feurs, Montbrison, Loi...",45.743563,4.226839
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,1 RUE HENRI RENAUDIN CHARLEVILLE-MEZIERES 8000,"(1, Rue Henri Renaudin, La Bosse d'Etion, Char...",49.773546,4.708829
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,2 RUE DE LYON EPINAY SUR SEINE 93800,"(Rue de Lyon, Orgemont, Épinay-sur-Seine, Sain...",48.953836,2.292919


In [17]:
#checking for null values
#vérification des valeurs nulles

df.isnull().sum()

address_id     0
address        0
city           0
postal_code    0
add            0
coordinates    0
latitude       0
longitude      0
dtype: int64

In [16]:
df

,address_id,address,city,postal_code,add,coordinates,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,318 CHE DE ROUMAGOUA LA CIOTAT 13600,"(Chemin de Roumagoua, La Fieloula, La Ciotat, ...",43.197927,5.607635
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,19 RUE DES DAMES SAINTE SAVINE 10300,"(Rue des Dames, Sainte-Savine, Troyes, Aube, G...",48.290905,4.045789
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,22 RUE AMIRAL GUEPRATTE LE CONQUET 29217,"(Rue Amiral Guépratte, Le Croaë, Le Conquet, B...",48.361340,-4.766800
3,4,6 BD DES ETINES,LE COTEAU,42120,6 BD DES ETINES LE COTEAU 42120,"(Boulevard des Etines, Le Coteau, Roanne, Loir...",46.021659,4.092120
4,5,35 AV DU 159EME RIA,BRIANCON,5100,BRIANCON 5100,"(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",44.898404,6.643631
...,...,...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,20 RUE COLI MONTREUIL 93100,"(20, Rue Coli, Ruffins - Théophile-Sueur, Mont...",48.868459,2.468415
558,602,6 AV JEAN JAURES,FEURS,42110,6 AV JEAN JAURES FEURS 42110,"(6, Avenue Jean Jaurès, Feurs, Montbrison, Loi...",45.743563,4.226839
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,1 RUE HENRI RENAUDIN CHARLEVILLE-MEZIERES 8000,"(1, Rue Henri Renaudin, La Bosse d'Etion, Char...",49.773546,4.708829
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,2 RUE DE LYON EPINAY SUR SEINE 93800,"(Rue de Lyon, Orgemont, Épinay-sur-Seine, Sain...",48.953836,2.292919


In [38]:
#sending table to database 
#envoi du tableau à la base de données

df.to_sql('addy', dbconnection, if_exists='replace', index = False, schema='dataengineer')

562